In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

from loaders import make_train_val_test_loaders, make_cv_loaders
from network import MicrostructureNet
from dataset import MicrostructurePatchDataset
from evaluate import evaluate_and_visualize_single_head
from train import train, train_single_val

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from pathlib import Path


In [ ]:
import torch
import os

# ---------------- Parameters ----------------
data_root = os.path.join("data_labeling", "src", "raw_images", "attention")

batch_size = 64
n_splits = 5
patch_size = 256
stride = 64

# List of LBP settings [(R, P), ...] or None
lbp_settings = [(1, 8), (3, 16)]  # your example
# lbp_settings = None  # if you want to disable LBP

# ---------------- Create CV loaders ----------------
cv_loaders = make_cv_loaders(
    data_root=data_root,
    batch_size=batch_size,
    n_splits=n_splits,
    patch_size=patch_size,
    stride=stride,
    lbp_settings=[(1, 8), (3, 16)]
)

# ---------------- Example usage ----------------
for fold, train_loader, val_loader in cv_loaders:
    print(f"Fold {fold}:")
    print(f"  Train batches: {len(train_loader)}")
    print(f"  Validation batches: {len(val_loader)}")

    # Peek at a single batch
    batch = next(iter(train_loader))
    print("  Batch keys:", batch.keys())
    print("  RGB shape:", batch["rgb"].shape)
    if batch["lbp"] is not None:
        print("  Number of LBP channels:", len(batch["lbp"]))
        print("  LBP patch shape:", batch["lbp"][0].shape)
    print("  Labels shape:", batch["label"].shape)

    # Only doing one fold for demonstration
    break


In [ ]:
train_loader, val_loader, test_loader = make_train_val_test_loaders(
    data_root,
    batch_size=128,
    patch_size=128,
    stride=64,
    lbp_settings=[(1, 8), (3, 16)],
    val_frac=0.15,
    test_frac=0.15,
    num_workers=0, # Windows Jupyter safe - multiprocessing issue
    random_state=42
)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
# ------------------ Model ------------------
model = MicrostructureNet(lbp_settings=lbp_settings, freeze_backbone=True).to(device)

# --------------- Loss + Optimizer-----------
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-3
)

In [ ]:
model, history = train_single_val(
    model,
    optimizer,
    criterion,
    train_loader,
    val_loader,
    device,
    num_epochs=50,
    patience=7,
)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

probs, coords = evaluate_and_visualize_single_head(
    model=model,
    loader=val_loader,
    device=device,
    max_show=32
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

# Canvas size (adjust as needed)
H, W = 100, 100

# Heatmap accumulation
heatmap = np.zeros((H, W), dtype=float)
countmap = np.zeros((H, W), dtype=float)

# Accumulate values
for (x1, y1, x2, y2), val in zip(coords, probs):
    heatmap[y1:y2, x1:x2] += val
    countmap[y1:y2, x1:x2] += 1

# Avoid division by zero
mask = countmap > 0
heatmap[mask] /= countmap[mask]

# Smooth overlaps
heatmap = gaussian_filter(heatmap, sigma=2)

# Plot
plt.figure(figsize=(6, 6))
plt.imshow(heatmap, cmap="inferno", origin="lower")
plt.colorbar(label="Value intensity")
plt.title("Bounding Box Heatmap")
plt.show()